# C&R

# 第４章 言語生成AI応用例


In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='ここに取得したOPENAI_API_KEYを入力')

In [ ]:
response = client.chat.completions.create(

    model="gpt-3.5-turbo",
    messages = [
        {"role": "system","content":"あなたは日本の株式会社の会計担当者です。"},
        {"role": "user", "content":"インボイスとはなんですか？"}]

)

##　AIの回答を追加

In [ ]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    # システム設定
    { "role": "system", "content": "あなたは日本の株式の会社の会計担当者です。"},
    # 最初の質問
    {"role": "user", "content": "インボイスとはなんですか？"},

    # 上記の質問に対するAIの回答と、それに対するユーザーの質問を追加する
    {"role" : "assistant", "content": "インボイスは、商品やサービスの提供に関連する請求書のことです。通常、販売者が顧客に対し、提供した商品やサービスの代金支払いを要求するために使用されます。インボイスには、販売者の連絡先情報、請求金額、支払い期限などが記載されています。また、企業間の取引では税金や関税の計算にも使用されることがあります。インボイスは販売者の売上や買い手の支出を正確に追跡するために重要な文書です。" },
    {"role": "user", "content": "つまり、どうすればいいですか？"},
    ]
)

## コードで追加していく

In [ ]:
from openai import OpenAI
# APIキーを設定
client = OpenAI(api_key = "" )#
# メッセージを記録するリストを用意し、最初にシステムロールを加える
messages = [
    {"role": "system", "content": "回答は50文字程度にする\n"}
]

# 以下、ユーザーがexitと入力するまで、会話履歴（コンテキスト）を追加していく
while True:
    # ユーザーに質問を促す
    user_prompt = input("質問を入力してください （'exit'を入力してenterキーを押すと会話を終了します）: ")

    # exit と入力された場合の処理（会話を終了）
    if user_prompt.lower() == 'exit':
    # ここまでのコンテキストをすべて出力する
        for message in messages:
            print(message)
        break

    # ユーザーの質問を role User としてコンテキストに追加保存
    messages.append({"role": "user", "content": user_prompt})

    # コンテキストを使って質問を送る
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # AIの回答
    model_response = completion.choices[0].message.content
    print(model_response)

    # アシスタントの回答を role assistant としてコンテキストに追加保存
    messages.append({"role": "assistant", "content": model_response})
    # exit が入力されるまで繰り返す



## Assistant API の利用手順
以下の手順で進めます。

1. Assistantの作成
2. Threadの作成
3. ThreadへのMessageの追加
4. Assistantの実行
5. Run Statusの確認
6. Responseの取得
7. ThreadとAssistantの削除（もう使わないのであれば）

### Assistantの作成

In [ ]:
# Assistantの作成
my_assistant = client.beta.assistants.create(
    name="とくしまリスキル",
    description="生成AIについて回答してくれるAI",
    model="gpt-4-1106-preview",
    instructions="あなたはAIについて詳しいアシスタントです。AIについて尋ねられたら高校生でもわかるように説明してください",
    tools=[{"type": "code_interpreter"}],
)
assistant_id = my_assistant.id
print(assistant_id)

### Threadの作成

Threadとは、会話ごとに用意される実行環境で、ここに会話履歴が自動的に保存されます。

In [ ]:
# Threadの作成
my_thread = client.beta.threads.create()
thread_id = my_thread.id
print(thread_id)

thread_BDVtf2wKpIaezt9OB1DQtEWM


### Message の作成

作成したThreadにメッセージをセットします。

In [ ]:
# Messageの作成
client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="生成AIを利用することで注意すべきことを100文字程度で教えて",
)

Message(id='msg_ZP3Lu0uRX4BaSftBoNgY6eyW', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='生成AIを利用することで注意すべきことを100文字程度で教えて'), type='text')], created_at=1712122068, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BDVtf2wKpIaezt9OB1DQtEWM')

### Assistantを実行する



In [ ]:
# Assistantの実行
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
)
run_id = run.id
print(run_id)

run_mxKZtu0iL5mMj3UjicoeWwu2


### Run ステータスの確認

いま実行した処理の状況(`run`)を表すステータス情報が返されるので確認する


In [ ]:
# 実行ステータスの確認
run_retrieve = client.beta.threads.runs.retrieve(
    thread_id=thread_id,
    # 実行処理のidを指定
    run_id=run_id,
)
print(run_retrieve.status)

completed


completed
と表示されれば、会話が成立しているので確認する。in progress の場合は、まだサーバーとのやり取りが終わっていない。

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread_id
)
print(messages.data)

[Message(id='msg_kAUFn74Za7zTMIoodim6wHmm', assistant_id='asst_tx8I6JM0YNw7RkwCqDIjhX3J', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='生成AIを利用する際は、生成されたコンテンツの信頼性や倫理的な側面に注意が必要です。データのプライバシーや機密性を保護し、バイアスや差別を排除するための適切な調整や監視が重要です。また、生成された情報の正確性を確認し、適切なコンテキストやガイドラインに従って使用することが重要です。'), type='text')], created_at=1712122122, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_mxKZtu0iL5mMj3UjicoeWwu2', status=None, thread_id='thread_BDVtf2wKpIaezt9OB1DQtEWM'), Message(id='msg_ZP3Lu0uRX4BaSftBoNgY6eyW', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='生成AIを利用することで注意すべきことを100文字程度で教えて'), type='text')], created_at=1712122068, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BDVtf2wKpIaezt9OB1DQtEWM')]


 会話部分だけを取り出す

In [ ]:
for message in messages.data:
    print(message.content[0].text.value)

生成AIを利用する際は、生成されたコンテンツの信頼性や倫理的な側面に注意が必要です。データのプライバシーや機密性を保護し、バイアスや差別を排除するための適切な調整や監視が重要です。また、生成された情報の正確性を確認し、適切なコンテキストやガイドラインに従って使用することが重要です。
生成AIを利用することで注意すべきことを100文字程度で教えて


対話を進める

In [ ]:
# Threadにコンテキストを追加
client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="信頼できるデータに基づいているかどうかを調べる方法を教えてください。",
)

Message(id='msg_YaeHRk2YAZeIAMXDNgIAAVBh', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='信頼できるデータに基づいているかどうかを調べる方法を教えてください。'), type='text')], created_at=1712122294, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BDVtf2wKpIaezt9OB1DQtEWM')

コンテキストが追加されたので、改めてThreadを実行する






In [ ]:
#Threadに改めて質問を送る
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
)
run_id = run.id
print(run_id)

run_G2Z9xtbhE64bdGuQmoK8hbSa


In [ ]:
# Runのステータスの確認
run_thread = client.beta.threads.runs.retrieve(
    thread_id=thread_id,
    run_id=run_id,
)
print(run_thread.status)

completed


completed となったら、メッセージを取り出し確認する。


In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread_id
)
print(messages.data)

[Message(id='msg_G9wowXV3bcewfzAcvgtgDcsO', assistant_id='asst_tx8I6JM0YNw7RkwCqDIjhX3J', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='信頼できるデータかどうかを確認するためには以下の方法が役立ちます。\n1.ソースの信頼性を確認する：公式なサイトや著名な機関からの情報かどうかを確認します。\n2.データの更新日時を確認する：最新の情報であるかどうかを確認します。\n3.複数の情報源を比較する：複数の情報源から同様の情報が得られるかどうかを確認します。\n4.専門家の意見を参考にする：専門家の見解を聞いて、データの信頼性を検証します。'), type='text')], created_at=1712122301, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_G2Z9xtbhE64bdGuQmoK8hbSa', status=None, thread_id='thread_BDVtf2wKpIaezt9OB1DQtEWM'), Message(id='msg_YaeHRk2YAZeIAMXDNgIAAVBh', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='信頼できるデータに基づいているかどうかを調べる方法を教えてください。'), type='text')], created_at=1712122294, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BDVtf2w

In [ ]:
for message in messages.data:
    print(message.content[0].text.value)
    print("-----------------------")

信頼できるデータかどうかを確認するためには以下の方法が役立ちます。
1.ソースの信頼性を確認する：公式なサイトや著名な機関からの情報かどうかを確認します。
2.データの更新日時を確認する：最新の情報であるかどうかを確認します。
3.複数の情報源を比較する：複数の情報源から同様の情報が得られるかどうかを確認します。
4.専門家の意見を参考にする：専門家の見解を聞いて、データの信頼性を検証します。
-----------------------
信頼できるデータに基づいているかどうかを調べる方法を教えてください。
-----------------------
生成AIを利用する際は、生成されたコンテンツの信頼性や倫理的な側面に注意が必要です。データのプライバシーや機密性を保護し、バイアスや差別を排除するための適切な調整や監視が重要です。また、生成された情報の正確性を確認し、適切なコンテキストやガイドラインに従って使用することが重要です。
-----------------------
生成AIを利用することで注意すべきことを100文字程度で教えて
-----------------------


逆順に取り出すと、メッセージの流れがわかる。

In [ ]:
for message in messages.data[::-1]:
    print(message.content[0].text.value)
    print("-----------------------")

生成AIを利用することで注意すべきことを100文字程度で教えて
-----------------------
生成AIを利用する際は、生成されたコンテンツの信頼性や倫理的な側面に注意が必要です。データのプライバシーや機密性を保護し、バイアスや差別を排除するための適切な調整や監視が重要です。また、生成された情報の正確性を確認し、適切なコンテキストやガイドラインに従って使用することが重要です。
-----------------------
信頼できるデータに基づいているかどうかを調べる方法を教えてください。
-----------------------
信頼できるデータかどうかを確認するためには以下の方法が役立ちます。
1.ソースの信頼性を確認する：公式なサイトや著名な機関からの情報かどうかを確認します。
2.データの更新日時を確認する：最新の情報であるかどうかを確認します。
3.複数の情報源を比較する：複数の情報源から同様の情報が得られるかどうかを確認します。
4.専門家の意見を参考にする：専門家の見解を聞いて、データの信頼性を検証します。
-----------------------


## Threadの削除とAssistantの削除



In [ ]:
thread_id = "thread_78gxUvv0rSsW0z9fxI9XtK7K"

In [ ]:
#  先に用意しておいた Threadのidを指定して削除
thread_deletion_status = client.beta.threads.delete(thread_id)
print(thread_deletion_status)

ThreadDeleted(id='thread_78gxUvv0rSsW0z9fxI9XtK7K', deleted=True, object='thread.deleted')


In [ ]:
assistant_id = "asst_WMDs4VTyQVWKVB7EJ8icxdYt"

In [ ]:
# 次にAssistantのIDを指定して削除
assistant_deletion_status = client.beta.assistants.delete(assistant_id=assistant_id)
print(assistant_deletion_status)


AssistantDeleted(id='asst_WMDs4VTyQVWKVB7EJ8icxdYt', deleted=True, object='assistant.deleted')


## 独自データの追加

###  Azure ハイブリッド検索

In [ ]:
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential

In [ ]:
import os
# Azure AI Search key
os.environ['SEARCH_ENDPOINT'] =  "https://****.search.windows.net"
os.environ['SEARCH_API_KEY'] = "yFzhBpnB****LAzSeB8ZfhP"

# Azure OpenAI key
os.environ['OPENAI_API_BASE'] = "https://*****.openai.azure.com/"
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
deployment_id_chat = "gpt4-1106-preview"
deployment_id_embedding = "text-embedding-ada-002"

In [ ]:
search_index_name = "vector-oyokiso"
semantic_configuration_name = search_index_name + "-semantic-configuration"


In [ ]:

client = AzureOpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
  api_version="2024-03-01-preview",
  azure_endpoint=os.environ['OPENAI_API_BASE']
)

In [ ]:
credential = AzureKeyCredential(search_api_key)

search_client = SearchClient(
    endpoint=os.environ['SEARCH_ENDPOINT'],#service_endpoint,
    index_name= search_index_name,
    credential=credential
)

In [ ]:
def get_embedding(question):
  response = client.embeddings.create(
     input=question,
     model=deployment_id_embedding
  )

embeddings = response.data[0].embedding

return embeddings

In [ ]:
def search(question):

  # AI Searchを検索する
  results = search_client.search(
    search_text=question,
    search_mode="any",
    search_fields=["chunk"],
    select=["title", "chunk"],
    semantic_configuration_name=semantic_configuration_name,
    top=5,

    vector_queries=[
       VectorizedQuery(
          kind = "vector",
          vector = generate_embedding(question),
          k_nearest_neighbors = 5,
          fields = "vector"
       )
    ]
  )


In [ ]:
def send_query(question):

   # Azure AI Searchを検索する
   results = search(question)

   # 応答データを取得する
   input_data = []
   for result in results:
      input_data.append(f"タイトル：{result['title']}\n")
      input_data.append(f"本文：{result['chunk']}\n\n")

   # GPTに回答をリクエスト
   system_message = '''
      資料「数理・データサイエンス・ＡＩ教育プログラム認定制度（応用基礎レベル）実施要綱細目」
      の内容をもとに、申請者からの質問に回答してください。
    '''

   for input in input_data:
      system_message = f"{system_message}{input}\n\n"

   messages = [{"role": "system", "content": system_message}]
   messages.append({"role": "user", "content": question})

   response = client.chat.completions.create(
      model = deployment_id_chat,
      messages = messages,
      temperature = 0.5
   )

   assistant_message = response.choices[0].message
   if assistant_message.content:
      messages.append({"role": assistant_message.role, "content": assistant_message.content})

   return messages

In [ ]:
res = send_query("高等学校は応募資格がありますか？")

In [ ]:
res[-1]['content']

## 情報抽出

In [ ]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
!wget https://infoart.ait231.tokushima-u.ac.jp/images/invoice.pdf

In [ ]:
# ファイルをOpenAIのサーバーにアップロード
file = client.files.create(
    file=open("/content/invoice.pdf", "rb"),
    purpose="assistants"
)

In [ ]:
# アシスタントの作成
assistant = client.beta.assistants.create(
    instructions="あなたは、指定されたドキュメント（ナレッジ）を調査して、情報を構造化し、その結果にもとづいて回答するChatBotです。",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="請求先、請求元、請求金額をそれぞれテーブル形式で出力してください？"
)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="請求先、請求元、請求金額をそれぞれテーブル形式で出力してください。"
)

In [ ]:
# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [ ]:
# 実行状況の確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

In [ ]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)

## 音声起こし

In [ ]:
# ffmpeg -i  C:/Users/ishida/Downloads/test.mp4 C:/Users/ishida/Downloads/test.mp3
# C:/Users/石田/ドキュメント > ffmpeg -i test.mp4 test.mp3

In [ ]:
%pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
model = whisper.load_model("large")

In [ ]:
result = model.transcribe("/content/レコーディング.m4a")
print(result["text"])

In [ ]:
import pandas as pd
df_transcribe = pd.DataFrame(result["segments"])
df_transcribe.head()

In [ ]:
result_translate = model.transcribe(audio="/content/レコーディング.m4a", task="translate", language="en")
df_translate = pd.DataFrame(result_translate["segments"])
df_translate.head()

### openai ライブラリ版

In [ ]:
%pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file, response_format="text")
print(transcript)

In [ ]:
translation = client.audio.translations.create(model="whisper-1", file=audio_file, response_format="text")
print(translation)

## Function Calling

In [ ]:
def add (a, b):
  return(a+b)

add(1,2)

In [ ]:
import requests
import json
# 関数を作成（関数を定義した後、その利用テストをしている）
def get_prefecture_code(pref_name):
    # 都道府県名と地域コードの対応辞書を用意
    pref_code_map = {
        '北海道': '01', '青森': '02', '岩手': '03', '宮城': '04',
        '秋田': '05', '山形': '06', '福島': '07', '茨城': '08',
        '栃木': '09', '群馬': '10', '埼玉': '11', '千葉': '12',
        '東京': '13', '神奈川': '14', '新潟': '15', '富山': '16',
        '石川': '17', '福井': '18', '山梨': '19', '長野': '20',
        '岐阜': '21', '静岡': '22', '愛知': '23', '三重': '24',
        '滋賀': '25', '京都': '26', '大阪': '27', '兵庫': '28',
        '奈良': '29', '和歌山': '30', '鳥取': '31', '島根': '32',
        '岡山': '33', '広島': '34', '山口': '35', '徳島': '36',
        '香川': '37', '愛媛': '38', '高知': '39', '福岡': '40',
        '佐賀': '41', '長崎': '42', '熊本': '43', '大分': '44',
        '宮崎': '45', '鹿児島': '46', '沖縄': '47'
    }
    # 入力された県名から「県」「都」「府」を削除
    for suffix in ['県', '都', '府']:
        if pref_name.endswith(suffix):
            pref_name = pref_name.replace(suffix, '')
            break
    # 地域コードを返す
    return pref_code_map.get(pref_name)


def fetch_weather_overview(pref_name):
    # 都道府県名から地域コードを取得
    pref_code = get_prefecture_code(pref_name)
    if not pref_code:
        return 'Invalid prefecture name'

    # 天気予報のAPIエンドポイント
    url = f"https://www.jma.go.jp/bosai/forecast/data/overview_forecast/{pref_code}0000.json"

    # リクエストを送信
    response = requests.get(url)

    # レスポンスをチェック
    if response.ok:
        # JSONデータとして解析
        data = response.json()
        # 'reportDatetime' と 'text' を結合して返す
        result = f"Report Datetime: {data['reportDatetime']}\nForecast Text: {data['text']}"
        return result
    else:
        # エラー情報を文字列で返す
        return 'Failed to fetch weather overview'

# 例
print(fetch_weather_overview('東京'))

In [ ]:

# 関数をいつ使うか指定
tools = [
    {
        "type": "function",
        "function": {
            # 関数の名前
            "name": "fetch_weather_overview",
            # 関数の説明
            "description": "日本の都道府県のお天気について、日付（何月何日）と地域、その地域の天気状況の説明文を返す関数です",
            "parameters": {
                "type": "object",
                "properties": {
                    # 関数に与える都道府県の名前
                    "pref_name": {
                        # 引数のデータ型を文字列と指定
                        "type": "string",
                        # 引数の例
                        "description": "東京都",
                    },
                },
                # 引数として都道府県名を指定する必要がある
                "required": ["pref_name"],
            },
        },
    }
]

```

関数名と使い方をまとめた`tools` をChatGPTに指定します。

```python
# 関数の使い方
messages = [{"role": "user", "content": "徳島県のいまの天気は?"}]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    # tools（関数）を指定する
    tools=tools,
    # 関数を実行する必要があるかどうかはChatGPTが判断
    tool_choice="auto",
)
```

このように設定すると ChatGPTは指定された関数を呼び出すタイミングと、呼び出し方を認識します。

```python
response_message = response.choices[0].message

print("用意された関数を呼び出す必要のある質問か？")
tool_calls = response_message.tool_calls
print(tool_calls)
print("今の質問に対する回答はあるか？")
print(response_message.content) #
print("関数を実行する条件")
print(response_message.tool_calls[0].function)
```

    用意された関数を呼び出す必要のある質問か？
    [ChatCompletionMessageToolCall(id='call_VVDEHYiylMXNBm10DydmTDr4', function=Function(arguments='{"pref_name":"徳島県"}', name='fetch_weather_overview'), type='function')]
    今の質問に対する回答はあるか？
    None
    関数を実行する条件
    Function(arguments='{"pref_name":"徳島県"}', name='fetch_weather_overview')

ただし、関数の実行そのものはユーザーの仕事です。ChatGPTの判断結果が`tool_calls` に入っています。これが真(`True`)であれば、ユーザーは関数を実行して、その結果を改めてChatGPTに与えます。


```python
# 関数の実行が必要と判断した場合、その関数を実行
if tool_calls:
    available_functions = {
        "fetch_weather_overview": fetch_weather_overview,
    }
    messages.append(response_message)
    # お天気状況について検索してから、再度OpenAIのAPIに回答文を生成させる
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            pref_name=function_args.get("pref_name"),
        )
        # 関数の実行結果をChatGPTに与える
        weather_str = ', '.join(function_response)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": weather_str,
            }
        )
    # 返り値をすべて確認するため、すべて出力
    print(messages)
    # ChatGPTは実行結果を参考に回答を生成
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
    )
print("------ 以下、ChatGPTの回答 ----------------")
print(second_response.choices[0].message.content)
